In [ ]:
import girder_client 
%load_ext autoreload
%autoreload 2
#import adrcSchemaHelpers as hlprs
import metadata_pipeline as mph 
import brain_region_maps as brm

#gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

In [ ]:
NeuroPathDemoImages_Collection='638e2da11f75016b81fda12f'

In [ ]:
mph.populateMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputFailed=False)

In [ ]:
mph.auditMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputRecords=True)

In [ ]:
mph.getSummaryStats(gc, collectionID=NeuroPathDemoImages_Collection)

In [ ]:
import pandas as pd

summaryStats = pd.read_csv("./summaryStats.csv", dtype=str)
print(summaryStats.head())

In [ ]:
def statSummary(row):
    
    filt = summaryStats["uid"] == row["uid"]
    summary = summaryStats.loc[filt, "state"].value_counts()
    
    for val in summary.index:
        row[f"{val}_count"] = summary[val]
        
    row["year"] = row["uid"][:4]
    row["case"] = row["uid"][4:]
    
    return row

In [ ]:
summaryStats["uid"] = summaryStats["year"] + summaryStats["folder_name"]

uids = pd.DataFrame({"uid": summaryStats["uid"].unique()})
uids[["year", "case", "invalid_count", "control_count", "valid_count"]] = None

uids = uids.apply(statSummary, axis=1)
uids.fillna(0.0, inplace=True)

uids.drop(columns=["uid"], inplace=True)

uids.columns = ["year", "case", "invalid_count", "control_count", "valid_count"]

uids.to_csv("./case_counts_by_slide_validity.csv", index=False)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
# girder_client docs are here: https://girder.readthedocs.io/en/stable/python-client.html#the-python-client-library
import girder_client

from dotenv import load_dotenv

import metadata_pipeline as mph 
import brain_region_maps as brm

In [ ]:
load_dotenv()
gc = girder_client.GirderClient(apiUrl=os.environ.get("dsaApiUrl"))
gc.authenticate(apiKey=os.environ.get("dsaApiToken"))

In [ ]:
NeuroPathDemoImages_Collection='63d98235e08050e0a7a8d75d'

In [ ]:
mph.populateMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputFailed=True)

In [ ]:
mph.auditMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputRecords=True)

In [ ]:
## Need to do this on a per case basis
needsBrainRegionMap = []
slideSummaryData = {}

for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
    year = yrf['name']
    if year.isdigit():
        slideSummaryData[year] = {} ## Going to create a stats dictionary ordered by year/pt
        
        for pt in gc.listFolder(yrf['_id']):
            slideSummaryData[year][pt['name']] = {}
            
            if pt['name'] not in brm.MAP:
                #print(year,pt['name'],"Needs Brain Region Map")
                slideSummaryData[year][pt['name']] = {"status": "incomplete", "reason": "needs brain region map"}
        
            else:

                ptItemData = mph.getFolderContents(gc,pt['_id']) ### Get all of the items cfor selected year
        
                stats = getPtSlideDataSummary(ptItemData,debug=True)
                slideSummaryData[year][pt['name']] = stats

In [ ]:
needsMap = []
for yr in slideSummaryData:
    for pt in slideSummaryData[yr]:
        caseMeta = slideSummaryData[yr][pt]
        if caseMeta['status'] == 'parsed':
            print(yr,pt,caseMeta['valid'],caseMeta['notValid'])
        elif caseMeta['status'] == 'incomplete':
            needsMap.append((yr,pt))

In [ ]:
mph.auditMetadata(gc,collectionID=NeuroPathDemoImages_Collection,outputRecords=True)

In [ ]:
# folder2020 = '638e81331f75016b81febf39'

# itemData2020 = mph.getFolderContents(gc,folder2020)

In [ ]:
allNpWorking = []

for itm in itemData2020:
    if 'meta' in itm and 'npWorking' in itm['meta']:
        allNpWorking.append(itm)

In [ ]:
[(x['meta']['npWorking'],x['name']) for x in allNpWorking]

In [ ]:
for pt in slideSummaryData['2017'].keys():
    s=  slideSummaryData['2017'][pt]
    if 'valid' in s:
        print('2017',pt,s['valid'],s['notValid'],s['controlSlides'])
    

In [ ]:
for i in slideSummaryData['2017']['E17-39']['slidesToReview']:
    print(i)

In [ ]:
allItemData = mph.get

In [ ]:
# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             print(ptId['name'])
#             hlprs.scanMetadata(gc,ptId,updateGirder=True)

In [ ]:
mph.getSummaryStats(gc, collectionID=NeuroPathDemoImages_Collection)

In [ ]:
mph.getMissingBrainRegion()